In [ ]:
#uses RegisteredMultiplier  - v1.0x

In [ ]:
import sys
import math
import io
import shutil

In [ ]:
import import_ipynb
from RegisteredNto1Adder_V1_1 import RegisteredNto1Adder

In [ ]:
def DotProduct_noHR_M(parameters, Print_To_File, Files_Location, Is_Top, modules):
    
    N = parameters["N"]
    AdderSize = parameters["ADDER_SIZE"]
    
    #IN_WIDTH>=1
    IN_WIDTH = 10
    #INPUT_REG_DEPTH>=0
    INPUT_REG_DEPTH = 1
    #MULT_PIPE_DEPTH>=0
    MULT_PIPE_DEPTH = 1

    ModuleName = "DotProduct_{}_noHR_M_A{}".format(N, AdderSize)

    if Is_Top:
        Files_Location += ModuleName+"/"
        if not os.path.exists(Files_Location):
            os.makedirs(Files_Location)

    if Print_To_File<=0:
        of=sys.stdout
    else:
        of=open(Files_Location+ModuleName+".v".format(N, AdderSize), 'w+')
        
    AdderSizes = set()

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
    print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
    print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    print("input inReady,", file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("A{}, ".format(i), end='', file=of)
    print(file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("B{}, ".format(i), end='', file=of)
    print(file=of)
    print("output outReady,", file=of)
    lgn=math.ceil(math.log2(N))
    al=-1+lgn
    if al==0:
        print("output signed [(2*IN_WIDTH):0] DP,", file=of)
    elif al<0:
        print("output signed [(2*IN_WIDTH){}:0] DP,".format(al), file=of)
    else:
        print("output signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
    print("output earlyOutReady", file=of)
    print(");\n", file=of)

    AIBuf=io.StringIO()

    level=0

    nextElememtNo=N #remained
    sizeList=[]
    for i in range(N):
        sizeList.append(1)

    RootList=[]
    for i in range(N):
        RootList.append(i)
    AdditionalRegList=[]
    for i in range(N):
        AdditionalRegList.append(0)

    i=0
    new = N

    while 1:
        #print("StartNextElememtNo = {}".format(nextElememtNo))
        if nextElememtNo==1:
            break
        if nextElememtNo<=AdderSize:
            lastLevel=True
        else:
            lastLevel=False        
        currentElememtNo = nextElememtNo
        nextElememtNo=0
        ORimp=False
        RegEnableImp=False
        while currentElememtNo>0:
            if currentElememtNo >= AdderSize:
                CurrentAdderSize = AdderSize
                currentElememtNo -= AdderSize
            else:
                if not lastLevel:
                    CurrentAdderSize = 1
                    currentElememtNo -= 1
                else:
                    CurrentAdderSize = currentElememtNo
                    currentElememtNo = 0
            nextElememtNo=nextElememtNo+1
            #print("nextElememtNo = {}".format(nextElememtNo))
            outSize=0
            for k in range (i, i+CurrentAdderSize):
                outSize+=sizeList[k]
            sizeList.append(outSize)
            if CurrentAdderSize==1: #just reg
                root=RootList[i]
                if root != -1:
                    RootList.append(i)
                    #print(root)
                    while root>=N:
                        root=RootList[root]
                        #print(root)
                    AdditionalRegList[root]+=1
                else:
                    RootList.append(-1)
                i+=1
                RegEnableImp=True
            else: #Generate a AdderSize Adder (maybe less size for last stage)
                RootList.append(-1)
                if not lastLevel:
                    if outSize==2: #outSize>=2
                        AIBuf.write("wire signed [2*IN_WIDTH:0] M{};\n".format(new))
                    else:
                        AIBuf.write("wire signed [2*IN_WIDTH+{}:0] M{};\n".format(math.ceil(math.log(outSize,2))-1, new))
                inSizeL = math.ceil(math.log(sizeList[i],2))
                #print("inSize = {}".format(inSize))
                rE = i+CurrentAdderSize
                eSizeList=[]
                finalRootList=[]
                for k in range (i, rE):
                    curSizeL=math.ceil(math.log(sizeList[k],2))
                    eSizeL=inSizeL-curSizeL
                    eSizeList.append(eSizeL)
                    #print("esize = {}".format(eSizeL))
                    root=RootList[k]
                    if root != -1:
                        while root>=N:
                            root=RootList[root]
                    else:
                        root=k
                    finalRootList.append(root)
                    if eSizeL!=0:
                        AIBuf.write("wire signed [2*IN_WIDTH")
                        if inSizeL!=1:
                             AIBuf.write("+{}".format(inSizeL-1, k))
                        if(root<N):
                            AIBuf.write(":0] DPpart{}e = ".format(root))
                        else:
                            AIBuf.write(":0] M{}e = ".format(root))
                        AIBuf.write("{{")
                        AIBuf.write("{}".format(eSizeL))
                        AIBuf.write("{")
                        if(root<N):
                            AIBuf.write("DPpart{}".format(root))
                        else:
                            AIBuf.write("M{}".format(root))
                        AIBuf.write("[2*IN_WIDTH".format(k))
                        if curSizeL==0:
                            AIBuf.write("-1")
                        elif curSizeL>1:
                            AIBuf.write("+{}".format(curSizeL-1))
                        AIBuf.write("]}},")
                        if(root<N):
                            AIBuf.write("DPpart{}".format(root))
                        else:
                            AIBuf.write("M{}".format(root))
                        AIBuf.write("};\n")
                AIBuf.write("Registered{}to1Adder_NIR #(.IN_WIDTH(2*IN_WIDTH".format(CurrentAdderSize))
                
                AdderSizes.add(CurrentAdderSize)
                
                curSizeL=math.ceil(math.log(sizeList[i],2))
                if inSizeL>0:
                    AIBuf.write("+{}".format(inSizeL))
                AIBuf.write("))\nAdd{}(\nclk, reset, enable,\n".format(new))
                if level==0:
                    AIBuf.write("MOR,\n")
                else:
                    AIBuf.write("ORAdd[{}],\n".format(level-1))
                for k in range (i, rE):
                    if(finalRootList[k-i]<N):
                        AIBuf.write("DPpart{}".format(finalRootList[k-i]))
                    else:
                        AIBuf.write("M{}".format(finalRootList[k-i]))
                    if(eSizeList[k-i]!=0):
                        AIBuf.write("e");
                    AIBuf.write(", ")
                AIBuf.write("\n")
                if lastLevel:
                    AIBuf.write("outReady,\n")
                else:
                    if ORimp==False:
                        AIBuf.write("ORAdd[{}],\n".format(level))
                        ORimp=True
                    else:
                        AIBuf.write("ORa{}, //not used\n".format(new))
                if lastLevel:
                    AIBuf.write("DP,\n".format(new))
                else:
                    AIBuf.write("M{},\n".format(new))
                if lastLevel:
                    AIBuf.write("earlyOutReady);\n\n")        
                else :
                    AIBuf.write("aeor{}); //not used\n\n".format(new))
                i+=CurrentAdderSize
            new+=1
        level+=1

    #print(RootList)
    #print(AdditionalRegList)

    print("wire signed [(2*IN_WIDTH)-1:0] ", end='', file=of)
    for i in range (N):    
        print("DPpart{}".format(i), end='', file=of)
        if i==N-1:
            print(";\n", file=of)              
        else:
            print(", ", end='', file=of)
    print("wire MOR;", file=of)
    print(file=of)
    for i in range (N):
        print("RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),", file=of)
        print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH", end='', file=of)
        IARD = AdditionalRegList[i];
        if IARD!=0:
             print("+{}".format(IARD), end='', file=of)
        print("),\n.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))", file=of)
        print("M{}(clk, reset, enable,".format(i), file=of)
        print("inReady,", file=of)
        print("A{}, B{},".format(i, i), file=of)
        if i==0:
            print("MOR,", file=of)
        else:
            print("MOR{}, //not used".format(i), file=of)
        print("DPpart{},".format(i), file=of)
        print("eMOR{}); //not used".format(i), file=of)
        print(file=of)

    if level>1:
        print("wire [0:{}] ORAdd;\n".format(level-2), file=of)
    print(AIBuf.getvalue(), end='',file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()

    shutil.copy("./RegisteredMultiplier  - SRL_Input - v1.01.v",Files_Location+"RegisteredMultiplier.v") #no parameter
    modules["RegisteredMultiplier"]="RegisteredMultiplier"
    
    temp_e_EIR = False
    if "ENABLE_INPUT_REGISTERS" in parameters:
        temp_e_EIR = True
        temp_EIR = parameters["ENABLE_INPUT_REGISTERS"]
    parameters["ENABLE_INPUT_REGISTERS"] = 0
    for AS in AdderSizes:
        parameters["N"] = AS 
        RegisteredNto1Adder(parameters, Print_To_File, Files_Location, 0, modules)
    if temp_e_EIR:
        parameters["ENABLE_INPUT_REGISTERS"] = temp_EIR
    parameters["N"] = N

    modules[ModuleName]="DotProduct_noHR_M"
    return ModuleName